# SKLOE OUT FILE READER
## Developed by Xiaoxian Guo and Zhichao Fang

Copyright &copy; 2018

### 简介

基于python3.6 (Anaconda) 环境开发，主要针对FDAS处理后的数据，进行进一步后处理，实现数据读入，数据处理，数据输出。

FDAS主要对单个工况数据进行处理，我们希望基于python开发进一步的后处理工具，提高工作效率，并提供python接口，可以实现更多的实时功能。

主要应用场景：

- 针对不同工况，同一通道的数据横向对比分析
- 标准化的批量报表自动生成
- 标准化的数据后处理，针对刚度试验，衰减试验
- 便捷的自定义数据处理
- 应用于自动校波、校风

...


目前程序托管于GitHub，主页为：https://github.com/XiaoxG/skloe_outfile


目前拥有以下功能：

1. *.out 文件读入功能
2. 打印文件基本信息到显示屏，txt文本，Excel文本 （包括：段数，通道数，采样频率，采样时间，前标定段信息，各个通道单位，通道名，系数，每段数据的采样点数，起止时间等）
3. 打印基本统计信息到显示屏，txt文本，Excel文本 （包括：最大值，最小值，方差，均值，点数等）
4. 输出数据至*.mat文件或*.dat文件
5. 通道单位修复 （原out文件通道单位不能超过4个字符）
6. 实型值转换
7. 简单的数据时域统计分析（基于Pandas）

正在开发功能：

- 波浪数据频、时域分析（基于WAFO）
- 校波文件读入与波浪自动对齐相位
- 静刚度试验模块
- 合并运动文件

待开发功能：

- 运动数据读入与自动对齐相位
- *.out 文件写入（或使用通用二进制文件）
- 标准化报表自动生成 （pdf,latex）
- 标准化波浪数据频域分析
- 标准化衰减试验模块

系统要求：

- Python >= 3.6.3 |Anaconda custom (64-bit)|
- Numpy >= 1.13
- Pandas >= 0.20.3
- Matplotlib >= 2.1.0
- Scipy >= 0.19.1

强烈推荐使用Anaconda 构建Python科学计算环境

Anaconda: https://www.anaconda.com

读入数据后，数据类型为pandas.DataFrame，基于pandas库的各种函数均可调用，请发挥你的想象力。

Pandas documents: https://pandas.pydata.org

后续版本将基于WAFO开发统计处理模块，包括时域分析，频谱转换，极值预报，统计模型检验，疲劳分析等。

强烈推荐WAFO模块（Matlab, Python）: http://www.maths.lth.se/matstat/wafo/

更多功能会陆续推出，欢迎大家在Github平台参与代码开发

如有任何问题请联系：xiaoxguo@sjtu.edu.cn

## 运行环境检测与数据读入

In [ ]:
# 读入skloe_out库，进行操作
from pydas.outio import *

# 导入必要的依赖库
import sys
import numpy as np
import pandas as pd
import matplotlib
import scipy

# 图片显示
%matplotlib inline

In [ ]:
# 检查python 运行环境
print('Python version ' + sys.version)
print('Numpy version ' + np.__version__)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)
print('Scipy version ' + scipy.__version__)

In [ ]:
# 读入.out文件
sd1 = OutFile('test.out')

读入文件时可以选择读入所有段或指定数据段，默认读入所有数据段

如需读入某一个特定数据段：

sd1 = Skloe_OutFile('test.out', s_seg=1) #读入第一段, 

注意，s_seg 从0开始计数，第0段为前标定段，第1段为第一段数据

## 数据基本信息显示

读入out文件后，sd1可以直接查看一些信息，例如：

In [ ]:
# 查看通道数
sd1.chN

这种方法，还可以查看以下相关信息：

- sd1.segN          数据段数
- sd1.fs            采样频率
- sd1.scale         数据类型 ‘prototype’ or 'model'
- sd1.filename      读入数据文件名
- sd1.date          数据采样日期 MM-DD
- sd1.data          数据（采用pandas.dataFrame 格式）
- sd1.chInfo        各个通道相关信息（通道号，通道名称，通道单位，通道转换系数）
- sd1.segInfo       各个数据段相关信息（段号，各段数据起止时间，采样数，采样时间，备注）
- sd1.seg_statictis 各段数据统计值显示（最大值，最小值，方程，均值）

In [ ]:
# 查看通道信息
sd1.chInfo

In [ ]:
# 查看各个数据段信息
sd1.segInfo

In [ ]:
# 直接查看数据 第零段前5行
sd1.data[0].head(5)

## 数据调用相关

由于数据格式采用pandas.DataFrame格式，可以十分方便的进行数据引用，行索引为时间，列索引为通道名

数据引用采用以下格式：

sd1.data[n]['channel name'] 

n 为第几段数据，后面直接引用通道名即可，通道名部分还可以给list，如['F.line1','wave1']同时提取多列指定数据，更加丰富的数据索引方法，参见pandas支持文档。

基于此，可以直接利用pandas库丰富的内建函数，实现简单的数理统计，以下为一些例子，请发挥你的想象力：

pandas documents online:http://pandas.pydata.org/pandas-docs/stable/

In [ ]:
#显示第1段数据所有通道平均值 
sd1.data[1].mean()

In [ ]:
#显示第1段数据相关数理统计信息
sd1.data[1].describe()

In [ ]:
# 数据显示
sd1.data[1]['wave.front'].plot()

## 数据导出

数据导出提供以下两个直接可以用的方法：

1. sd1.to_dat(s_seg = 'all') 默认导出所有段，或可导出指定段
2. sd1.to_mat(s_seg = 0) 这里请输入正整数，导出为包含该段所有信息的.mat文件

以上两种方法，生成的文件均保存在运行文件目录下。

除此之外，还可直接利用pandas函数导出特定段，特定通道数据，例如:

sd1.data[0]['F.line1'].to_csv('sd1.dat') # 导出第0段F.line1到*.dat文件

这种情况下，可以转存数据到TXT，EXCEL，HTML...

数据段信息也可导出或查看，提供以下直接可用的方法(可以打印到显示屏，TXT，EXCEL(下一版本会支持打印到粘贴版)):

1. sd1.pInfo(printTxt=False,printExcel=False) 打印段落信息
2. sd1.pChInfo(printTxt=False,printExcel=False)打印通道信息
3. sd1.pst(printTxt=False,printExcel=False)打印统计值信息


In [ ]:
# 导出到*.dat
sd1.to_dat(s_seg = 0)

# 导出到*.mat
sd1.to_mat(s_seg = 0)

## 实型值数据转换

目前可以根据输入各个通道单位，实现实型值转换，步骤如下：

1. 请检查读取单位是否有误，原out文件单位仅能占4个字节，故超过四个字节单位要修复，如deg/s2，kg.cm等

    检查通道：sd1.pChInfo()
    
    检查是否已经转换：sd1.scale

2. 修复单位，请用以下方法：sd1.fix_unit(c_chN=['Ch36','Ch35'],unit=['kg.cm','kg.cm']) #修改36，37通道单位为kg.cm
    
3. 转换数据到实型值：sd1.to_fullscale()

注意，一经转换所有信息将变为实型值，以上提到的针对数据处理的方法均可用，通道转换可识别所有常用单位。

In [ ]:
sd1.fix_unit(c_chN=['Ch36','Ch35'],unit=['kg.cm','kg.cm'])

In [ ]:
sd1.to_fullscale()

In [ ]:
sd1.pChInfo()